In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device('cpu')

In [3]:
# read in all the words
words = open('input.txt', 'r').read().split()
words = [
    w
    for w in words
]
words[:8]

['प्रगतिशील', 'कर्मिक', 'और', 'कृषक', 'संघ', 'सूरीनाम', 'का', 'एक']

In [4]:
len(words)

146281

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'ँ', 2: 'ं', 3: 'ः', 4: 'अ', 5: 'आ', 6: 'इ', 7: 'ई', 8: 'उ', 9: 'ऊ', 10: 'ऋ', 11: 'ऍ', 12: 'ऎ', 13: 'ए', 14: 'ऐ', 15: 'ऑ', 16: 'ओ', 17: 'औ', 18: 'क', 19: 'ख', 20: 'ग', 21: 'घ', 22: 'ङ', 23: 'च', 24: 'छ', 25: 'ज', 26: 'झ', 27: 'ञ', 28: 'ट', 29: 'ठ', 30: 'ड', 31: 'ढ', 32: 'ण', 33: 'त', 34: 'थ', 35: 'द', 36: 'ध', 37: 'न', 38: 'ऩ', 39: 'प', 40: 'फ', 41: 'ब', 42: 'भ', 43: 'म', 44: 'य', 45: 'र', 46: 'ऱ', 47: 'ल', 48: 'ळ', 49: 'ऴ', 50: 'व', 51: 'श', 52: 'ष', 53: 'स', 54: 'ह', 55: '़', 56: 'ऽ', 57: 'ा', 58: 'ि', 59: 'ी', 60: 'ु', 61: 'ू', 62: 'ृ', 63: 'ॄ', 64: 'ॅ', 65: 'ॆ', 66: 'े', 67: 'ै', 68: 'ॉ', 69: 'ॊ', 70: 'ो', 71: 'ौ', 72: '्', 73: 'ॐ', 74: '।', 75: '॥', 76: '०', 77: '१', 78: '२', 79: '३', 80: '४', 81: '५', 82: '६', 83: '७', 84: '८', 85: '९', 86: '॰', 0: '.'}
87


In [6]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([641136, 3]) torch.Size([641136])
torch.Size([79667, 3]) torch.Size([79667])
torch.Size([79815, 3]) torch.Size([79815])


In [7]:
# MLP revisited
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5) #* 0.2
#b1 = torch.randn(n_hidden,                        generator=g) * 0.01
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.01
b2 = torch.randn(vocab_size,                      generator=g) * 0

# BatchNorm parameters
bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))
bnmean_running = torch.zeros((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))

parameters = [C, W1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

24757


In [8]:
# same optimization as last time
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
  
  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  # Linear layer
  hpreact = embcat @ W1 #+ b1 # hidden layer pre-activation
  # BatchNorm layer
  # -------------------------------------------------------------
  bnmeani = hpreact.mean(0, keepdim=True)
  bnstdi = hpreact.std(0, keepdim=True)
  hpreact = bngain * (hpreact - bnmeani) / bnstdi + bnbias
  with torch.no_grad():
    bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
    bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi
  # -------------------------------------------------------------
  # Non-linearity
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())
  

      0/ 200000: 4.4533
  10000/ 200000: 2.2798
  20000/ 200000: 2.5815
  30000/ 200000: 2.0368
  40000/ 200000: 2.0618
  50000/ 200000: 2.1470
  60000/ 200000: 2.0158
  70000/ 200000: 1.6857
  80000/ 200000: 2.6747
  90000/ 200000: 2.0864
 100000/ 200000: 1.7540
 110000/ 200000: 2.1887
 120000/ 200000: 1.8927
 130000/ 200000: 2.1198
 140000/ 200000: 1.9069
 150000/ 200000: 2.3750
 160000/ 200000: 2.2218
 170000/ 200000: 1.8186
 180000/ 200000: 2.0786
 190000/ 200000: 1.8252


In [9]:
plt.figure(figsize=(20,10))
# plt.imshow(h.abs()>0.99 ,cmap ='gray' ,interpolation='nearest')

<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>

In [10]:
# plt.hist(h.view(-1).tolist(),50)

In [11]:
# plt.plot(lossi)

In [12]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 # + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnstd = hpreact.std(0, keepdim=True)

In [13]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 # + b1
  #hpreact = bngain * (hpreact - hpreact.mean(0, keepdim=True)) / hpreact.std(0, keepdim=True) + bnbias
  hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')

split_loss('val')

train 1.9232988357543945
val 1.9440572261810303


In [28]:
import torch
import torch.nn.functional as F

# Set the generator seed
# g = torch.Generator().manual_seed(2147483647 + 10)

# Function to generate text
def generate_text(seed_text, num_words=8):
    for _ in range(num_words):
        # Prepare the context
        context = [0] * block_size
        for ch in seed_text[-block_size:]:
            context = context[1:] + [stoi.get(ch, 0)]
        
        out = []
        while True:
            # Forward pass
            emb = C[torch.tensor([context])] 
            embcat = emb.view(emb.shape[0], -1) 
            hpreact = embcat @ W1
            hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
            h = torch.tanh(hpreact)
            logits = h @ W2 + b2
            probs = F.softmax(logits, dim=1)
            # Sample from the distribution
            ix = torch.multinomial(probs, num_samples=1).item()
            context = context[1:] + [ix]
            out.append(ix)
            # Break if we sample the special '.' token
            if ix == 0:
                break
        
        word = ''.join(itos[i] for i in out[:-1])  # Exclude the '.' token
        seed_text += word + ' '
    
    return seed_text.strip()

# Array of Hindi words
hindi_words = ['नमस्ते', 'संपर्क', 'अधिक', 'अनुभव', 'कृपया', 'धन्य']

# Loop through the array and generate text for each word
for word in hindi_words:
    # print(f"\nStarting seed text: {word}")
    
    # Generate text
    generated_text = generate_text(word)
    
    # Print the generated text
    print(f"Generated text from '{word}' ---> {generated_text}")


Generated text from 'नमस्ते' ---> नमस्ते सकती राज्य हृर फिल्म बिन्न तेस झक्ष
Generated text from 'संपर्क' ---> संपर्कब वैफस्तर तनाने  र्ग  हैं। तो
Generated text from 'अधिक' ---> अधिकारा देता डिजि लीटुद्ध सूहिंदिर णीकों फीट
Generated text from 'अनुभव' ---> अनुभववळहन  अब द धकोशीय  अधिकार श
Generated text from 'कृपया' ---> कृपयागत वालींचों जनाते क ने  संख्या तान
Generated text from 'धन्य' ---> धन्ययनू तेल पूर्व शिक्स थती  अपसायस कृत
